# **Data science internship assignement - Dealroom.co**
*Done by Océane Salmeron, December 2020*

This notebook is for explaination only, therefore it will not perfom the excel export.

## 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

## 2. Import Data

In [2]:
def load_data(file):
    data = pd.ExcelFile(file)
    return data.parse('Data')

In [3]:
data = load_data('../Data/Data_Science_Internship_Assignment.xlsx')

Let's take a look at our raw data, and get more information about it with .info() and isna()

In [4]:
data.head(2)

,NAME,WEBSITE,TAGLINE,HQ REGION,HQ COUNTRY,HQ CITY,TAGS,LAUNCH DATE,GROWTH STAGE,LINKEDIN,TYPE
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,London,mobile,"2002, September",late growth stage,NaN,NaN
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,Birmingham,NaN,2008,early growth stage,https://www.linkedin.com/company/-futsal-group...,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11582 entries, 0 to 11581
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NAME          11582 non-null  object 
 1   WEBSITE       11582 non-null  object 
 2   TAGLINE       11453 non-null  object 
 3   HQ REGION     11582 non-null  object 
 4   HQ COUNTRY    11582 non-null  object 
 5   HQ CITY       10908 non-null  object 
 6   TAGS          9593 non-null   object 
 7   LAUNCH DATE   11582 non-null  object 
 8   GROWTH STAGE  8584 non-null   object 
 9   LINKEDIN      8596 non-null   object 
 10  TYPE          0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 995.5+ KB


In [6]:
data.isna().sum()

NAME                0
WEBSITE             0
TAGLINE           129
HQ REGION           0
HQ COUNTRY          0
HQ CITY           674
TAGS             1989
LAUNCH DATE         0
GROWTH STAGE     2998
LINKEDIN         2986
TYPE            11582
dtype: int64

Our data has 11582 entries and 11 columns. For this task, only **'TAGLINE'**,**'TAGS'**,**'LAUNCH DATE'** columns will be useful to classify the data. All the other columns are not relevant.

We can see that the **'LAUNCH DATE'** column is not of type datetime64 but object, we're gonna change that in part 2.

**'TAGLINE'** and **'TAGS'** columns contain some NaN values, we will also take care of that in part 2.

## 2. Cleaning the Data

First let's drop the column that are not relevant to our task.

In [7]:
df = data.drop(['WEBSITE', 'HQ REGION', 'HQ COUNTRY', 'HQ CITY', 'GROWTH STAGE', 'LINKEDIN'], axis=1)

**'TAGLINE'** and **'TAG'** columns both contains text data in different ways. **'TAGS'** is a list of tags separated by ';', whereas **'TAGLINE'** is a small description of the company.

For both columns, we're gonna transform them into list in order to be able to use them in our classify function. But first the text needs to be cleaned.

We will create a general function to convert the text to String, remove any numbers and transform it to lower caps.

In [8]:
def clean_text(x):
    x = x.astype(str)
    x = x.str.replace('\d+', '')
    x = x.str.lower() 
    return x

Then we will define 2 functions :

    1. One to transform the sequence into a list
    2. One to continue to clean 'TAGLINE'

In [9]:
def clean_tag(x):
    x = clean_text(x)
    # Create list from sequence
    x = x.str.split(pat=';')
    return x

To continue **'TAGLINE'** we will remove any kind of punctuations and then create a list from the sequence. After this we will remove any stopwords from the list we just created.

In [10]:
def clean_tagline(x):
    x = clean_text(x)
    # Remove punctuations
    x = x.str.replace(r'[^\w\s]','')
    x = x.str.strip()
    # Remove stop words
    stop_words = stopwords.words('english')
    x = x.apply(lambda i: [item for item in i.split() if item not in stop_words])
    return x

In [11]:
def remove_duplicates(x):
    return list(dict.fromkeys(x))

In **Part 1** we saw that **'TAGLINE'** and **'TAGS'** both contains nan value. Due to a bug in .astype() function, NaN value from the excel are converted to 'nan'. For this reason we will remove them.

In [12]:
def clean_data(data):
    
    # Clean date
    data.rename(columns={'LAUNCH DATE':'LAUNCH_DATE'}, inplace=True)
    ## Convert date string to date format and only keep the year
    data['LAUNCH_DATE'] = pd.to_datetime(data['LAUNCH_DATE'].astype(str), errors="coerce").dt.year

    # Clean TAGS column
    data['TAGS'] = clean_tag(data['TAGS'])
    data.loc[data['TAGS']=='nan', 'TAGS'] = data.loc[data['TAGS']=='nan', 'TAGS'].apply(lambda x: [])
    
    # Clean TAGLINE column
    data['TAGLINE']= clean_tagline(df['TAGLINE'])
    data.loc[data['TAGLINE']=='nan', 'TAGLINE'] = data.loc[data['TAGLINE']=='nan', 'TAGLINE'].apply(lambda x: [])
    
    return data

In [13]:
df = clean_data(df)

Because we want to use both **'TAGLINE'** and **'TAGS'**, we will concatenate both list into one column

In [14]:
df['ALL']=df['TAGS']+df['TAGLINE']
df['ALL']=df['ALL'].apply(remove_duplicates)

# 3. Classification

In order to classify our data, we need lists of keywords for each entity type. We will build that list by a quick analyse from the most common words in **'ALL'**

In [16]:
keywords = list(df['ALL'].explode().values)
keywords= Counter(keywords)
keywords.most_common()

[('nan', 2073),
 ('software', 1978),
 ('media', 1165),
 ('mobile', 1161),
 ('design', 1138),
 ('data', 1060),
 ('services', 1044),
 ('company', 1036),
 ('solutions', 1014),
 ('platform', 947),
 ('subscription', 882),
 ('online', 804),
 ('finance', 797),
 ('social', 795),
 ('deep tech', 740),
 ('management', 703),
 ('uk', 669),
 ('advertising', 630),
 ('service', 628),
 ('business', 628),
 ('technology', 627),
 ('search engine', 597),
 ('digital', 567),
 ('provider', 548),
 ('cloud technology', 523),
 ('marketing', 512),
 ('video', 509),
 ('saas', 504),
 ('agency', 499),
 ('app', 495),
 ('leading', 494),
 ('retail', 477),
 ('community', 469),
 ('adtech', 468),
 ('delivery', 468),
 ('branding', 467),
 ('cleantech', 431),
 ('development', 429),
 ('content', 423),
 ('local', 410),
 ('e-commerce', 394),
 ('products', 392),
 ('fintech', 389),
 ('based', 383),
 ('home', 382),
 ('commission', 368),
 ('navigation', 364),
 ('web', 363),
 ('consulting services', 349),
 ('hardware', 345),
 ('risk'

In [17]:
startup=['software', 'mobile', 'design', 'data', 'deep tech', 'search engine', 'cloud technology', 'saas', 'video',
          'adtech', 'app', 'cleantech', 'e-commerce', 'fintech', 'regtech compliance', 'consulting services', 'hardware',
          'online', 'monitoring', 'social media', 'analytics', 'game', 'technology', 'enterprise software',
          'tech', 'it', 'wireless technology', 'developer tools', 'seo', 'data analytics', 'imaging technology',
          'machine', 'deep', 'artificial', 'solutions', 'startup','services']
    
education=['research','educational','student', 'university', 'school', 'certification', 
            'e-learning', 'study', 'studies', 'tutorials', 'academic','assesment','academics', 'learning',
            'skills', 'teach', 'teacher', 'education']
    
government=['charity','medical', 'healthcare','profit','non-profit','volunteer', 'volunteering', 
            'governmental','governement', 'organisation']
    
mature = ['mature', 'mutlinational', 'established', 'leader', 'leading']

To classify the data we will compute the length of the intersection set between the company keywords set and each entity type keywords set. 

The type accorded will be from the longer intersection set.

If the length of the maximum equals 0, meaning no keywords match with any entity type keywords, the company will be classed "Unclassified".

If the maximum is a Startup, we will perform a launch year check to classify it as a "Startup" if after 1990, otherwise as "Mature company".

In [18]:
def classify(x, tech, education, government, mature):
    
    dic = {'Startup' : len(set(x['ALL'])&set(tech)),
            'Universities/Schools' : len(set(x['ALL'])&set(education)),
            'Government/Non-profit' : len(set(x['ALL'])&set(government)),
            'Mature company' : len(set(x['ALL'])&set(mature))
            }
    
    maximum = max(dic, key=dic.get)
    entity = str(maximum)
    
    if(dic[maximum] == 0):
        entity = 'Unclassified'
    elif (maximum == 'Startup'):
        if (x['LAUNCH_DATE']<1990):
            entity = 'Mature company'
            
    return entity   

In [19]:
df['TYPE']=df.apply(classify, args=(startup, education, government, mature), axis=1)

In [20]:
print(df['TYPE'].value_counts())

Startup                  7544
Unclassified             3338
Mature company            277
Universities/Schools      269
Government/Non-profit     154
Name: TYPE, dtype: int64
